### Installs and Upgrades

need to do the following installations and updates

    apt-get update
    apt-get upgrade
    pip uninstall -y langchain langchain-community langchain-experimental langchain-core langchain-aws langgraph
    pip install "langchain-core>=0.3.27,<0.4.0"
    pip install "langchain>=0.3.13"
    pip install "langchain-community>=0.3.13"
    pip install "langgraph>=0.0.25"
    pip install boto3
    pip install wikipedia-api

### Imports

In [ ]:
from typing import List, Dict, Any
from langchain_community.chat_models.bedrock import BedrockChat
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import Graph, StateGraph
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import wikipedia
import boto3

### FactChecker Utility Functions

In [ ]:
class FactCheckAgent:

    def __init__(self):

        # create a callback manager
        callbacks = CallbackManager([StreamingStdOutCallbackHandler()])

        # initialize the Bedrock client
        bedrock_client = boto3.client(
            service_name='bedrock-runtime',
            region_name='us-east-1'
        )

        self.llm = BedrockChat(
            model_id="anthropic.claude-3-sonnet-20240229-v1:0",
            client=bedrock_client,
            callback_manager=callbacks,
            model_kwargs={
                "temperature": 0,
                "max_tokens": 4096,
                "anthropic_version": "bedrock-2023-05-31"
            }
        )


    def extract_fact_claims(self, text: str) -> List[str]:
        """Extract potential fact claims from text"""
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a precise fact-checking assistant. 
            Extract ONLY the specific factual claims from the input text.
            Return each claim on a new line.
            DO NOT include any additional text or formatting.
            DO NOT include your own commentary or explanations.
            ONLY list the actual claims from the input text."""),
            ("human", "{text}")
        ])

        chain = prompt | self.llm
        response = chain.invoke({"text": text})
        # filter out empty strings and strip whitespace
        claims = [claim.strip() for claim in response.content.split('\n') if claim.strip()]
        # additional filtering to remove any meta-text
        claims = [claim for claim in claims if not claim.startswith(("Here are", "These are", "The following"))]
        return claims

    def verify_with_llm_knowledge(self, claim: str) -> Dict[str, Any]:
        """verify claim using LLM's built-in knowledge"""
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a precise fact-checking assistant. Using ONLY your built-in knowledge (not external sources):
            1. Analyze the claim's accuracy
            2. Determine if it's TRUE, FALSE, or UNCERTAIN
            3. Provide a brief explanation
            4. Rate your confidence level on a scale of 1-5 (5 being highest)

            Format your response exactly as:
            STATUS: [TRUE/FALSE/UNCERTAIN]
            CONFIDENCE: [1-5]
            EXPLANATION: [Your explanation]"""),
            ("human", f"Claim: {claim}")
        ])

        chain = prompt | self.llm
        result = chain.invoke({})

        return {
            "claim": claim,
            "verification": result.content
        }

    def _generate_search_terms(self, claim: str) -> str:
        """Generate better search terms for Wikipedia lookup"""
        """This is optional and can use LLM or NLP to generate these automatically to improve results."""
        """Hardcoded here for the sake of POC"""
        keywords = {
            "World War II": ["World War II", "Second World War", "WW2"],
            "Shakespeare": ["William Shakespeare", "Shakespeare biography"],
            "Einstein": ["Albert Einstein", "Einstein physics"],
            "Great Wall of China": ["Great Wall of China", "Great Wall visibility from space"],
        }

        search_terms = claim
        for key, terms in keywords.items():
            if key.lower() in claim.lower():
                search_terms = terms[0]  # Use the first (most general) term
                break

        return search_terms

    def _wiki_fact_check(self, claim: str) -> Dict[str, Any]:
        """Wikipedia fact-checking logic"""
        try:
            search_terms = self._generate_search_terms(claim)
            search_results = wikipedia.search(search_terms)

            if search_results:
                try:
                    for result in search_results[:3]:
                        try:
                            page = wikipedia.page(result, auto_suggest=False)
                            content = page.summary
                            break
                        except wikipedia.exceptions.DisambiguationError as e:
                            try:
                                page = wikipedia.page(e.options[0], auto_suggest=False)
                                content = page.summary
                                break
                            except:
                                continue
                        except:
                            continue
                    else:
                        return {
                            "claim": claim,
                            "verification": "STATUS: UNCERTAIN\nEXPLANATION: Could not find reliable Wikipedia information to verify this claim.",
                            "source": None
                        }

                    verification_prompt = ChatPromptTemplate.from_messages([
                        ("system", """
                        You are a precise fact-checking assistant. Follow this protocol:
                        1. Carefully analyze the claim against the provided Wikipedia content
                        2. Determine if the claim is TRUE, FALSE, or UNCERTAIN
                        3. Provide a clear, concise explanation with specific references
                        4. Format your response as:
                           STATUS: [TRUE/FALSE/UNCERTAIN]
                           EXPLANATION: [Your explanation]
                        5. Keep explanations concise and focused on the specific claim
                        """),
                        ("human", f"Claim: {claim}\n\nWikipedia Content: {content}")
                    ])

                    verification_chain = verification_prompt | self.llm
                    result = verification_chain.invoke({})

                    return {
                        "claim": claim,
                        "verification": result.content,
                        "source": page.title
                    }

                except Exception as e:
                    return {
                        "claim": claim,
                        "verification": f"STATUS: UNCERTAIN\nEXPLANATION: Error accessing Wikipedia content: {str(e)}",
                        "source": None
                    }

            return {
                "claim": claim,
                "verification": "STATUS: UNCERTAIN\nEXPLANATION: No reliable Wikipedia source found to verify this claim.",
                "source": None
            }

        except Exception as e:
            return {
                "claim": claim,
                "verification": f"STATUS: ERROR\nEXPLANATION: {str(e)}",
                "source": None
            }

    def fact_check_claim(self, claim: str) -> Dict[str, Any]:
        """Fact-check a claim using both LLM knowledge and Wikipedia"""
        # get LLM knowledge-based verification
        llm_verification = self.verify_with_llm_knowledge(claim)

        # get Wikipedia-based verification
        wiki_verification = self._wiki_fact_check(claim)

        return {
            "claim": claim,
            "llm_verification": llm_verification["verification"],
            "wiki_verification": wiki_verification["verification"],
            "wiki_source": wiki_verification["source"]
        }

### Build LangGraph Workflow

In [ ]:
def create_fact_checking_workflow():

    workflow = StateGraph(dict)

    fact_check_agent = FactCheckAgent()

    def extract_claims(state: dict):
        """Extract claims from input text"""
        text = state.get('text', '')
        claims = fact_check_agent.extract_fact_claims(text)
        return {"claims": claims}

    def verify_claims(state: dict):
        """Verify extracted claims"""
        claims = state.get('claims', [])
        fact_checks = [fact_check_agent.fact_check_claim(claim) for claim in claims]
        return {"fact_checks": fact_checks}

    def generate_report(state: dict):
        fact_checks = state.get('fact_checks', [])
        report = "📋 Fact Check Report\n" + "="*50 + "\n\n"

        for i, check in enumerate(fact_checks, 1):
            report += f"Claim #{i}: {check['claim']}\n"
            report += f"{'='*30}\n"
            report += "🤖 LLM Knowledge Verification:\n"
            report += f"{check['llm_verification']}\n\n"
            report += "📚 Wikipedia Verification:\n"
            report += f"{check['wiki_verification']}\n"
            report += f"Source: {check['wiki_source'] or 'No specific source'}\n"
            report += f"{'-'*50}\n\n"

        return {"report": report, "completed": True}

    # add nodes
    workflow.add_node("extract_claims", extract_claims)
    workflow.add_node("verify_claims", verify_claims)
    workflow.add_node("generate_report", generate_report)

    # add edges
    workflow.set_entry_point("extract_claims")
    workflow.add_edge("extract_claims", "verify_claims")
    workflow.add_edge("verify_claims", "generate_report")

    # define end condition
    def check_end(state):
        return "end" if state.get("completed", False) else "generate_report"

    workflow.add_conditional_edges(
        "generate_report",
        check_end,
        {
            "end": "end",
            "generate_report": "generate_report"
        }
    )

    # add end node
    workflow.add_node("end", lambda x: x)

    return workflow.compile()

### Main Driver Function

In [ ]:
def main():

    workflow = create_fact_checking_workflow()

    text = """
    World War II happened in the 15th century. 
    Shakespeare was a famous painter from England. 
    Albert Einstein developed the theory of relativity.
    The Great Wall of China is visible from space.
    Paris is a city in Italy.
    Shahrukh Khan was the lead actor in the movie Sholay
    """

    result = workflow.invoke({"text": text})
    print(result['report'])

In [ ]:
if __name__ == "__main__":
    main()